# UCLA ITS Data Camp, Day 2

### Warm-Up / Review Exercise: Get Scooter Data
You may have seen some scooters around UCLA. What you may not have known is that the data on dockless mobility in Los Angeles is made publicly available via APIs provided by each company, which you can read more about [here](https://medium.com/tim-black/scooters-mapped-55a2afca46b1). Each company provides a separate URL for each city they operate in. I've compiled a master list of the scooter companies' GBFS feeds [here](https://github.com/black-tea/swarm-of-scooters/blob/master/data/systems.csv); for example, you will see that the Los Angeles region includes 7 different feeds across 6 providers. 

Using our new skills at getting data from APIs, let's practice grabbing data from these companies. Build a function that will take in a URL as an argument and will output a JSON file with the response content. 

_Function input:_ URL of GBFS data feed  
_Function output:_ Writes JSON to `data/raw` with filename format: `scooter_[%Y-%m-%d-%H-%M-%S].json` (no return value)

In [47]:
# TODO: Build a function that takes in a url and writes out the data to your `data/raw` directory.
#       Hint: Don't forget to import the necessary packages!

import requests
import json
import time
from datetime import datetime

def get_scooter_gbfs(scooter_url): 
    resp = requests.get(scooter_url)
    
    if resp.status_code == requests.codes.ok:
        with open(f'data/raw/scooter_[{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}].json', 'w') as outfile:
            json.dump(resp.json(), outfile)
    
    else:
        resp.raise_for_status()

Pick any of the URLs on [this page](https://github.com/black-tea/swarm-of-scooters/blob/master/data/systems.csv) to test out your function below.

In [48]:
# TODO: Add URL
scooter_url = 'https://mds.bird.co/gbfs/los-angeles/free_bikes'

# Run your function to download the data and store for later
get_scooter_gbfs(scooter_url)

In [49]:
import json
with open('data/raw/scooter_[2019-09-17-10-05-59].json') as json_file:
    bird_la = json.load(json_file)

In [50]:
bird_la

{'last_updated': 1568739959,
 'data': {'bikes': [{'bike_id': '9d5d5fc6-aeb7-4f35-85fa-990d59ab6cfe',
    'lat': 34.024936,
    'lon': -118.193322,
    'is_reserved': 0,
    'is_disabled': 0},
   {'bike_id': '6b57b39f-4d46-4a93-8a1c-d2b41cba0669',
    'lat': 34.018259,
    'lon': -118.309285,
    'is_reserved': 0,
    'is_disabled': 0},
   {'bike_id': '084f9800-891a-4e85-8c6b-e2fbc3eb505d',
    'lat': 34.094614,
    'lon': -118.338552,
    'is_reserved': 0,
    'is_disabled': 0},
   {'bike_id': 'eafa998c-41a3-427c-86db-822789b6b17c',
    'lat': 34.065551,
    'lon': -118.442585,
    'is_reserved': 0,
    'is_disabled': 0},
   {'bike_id': '931f5b50-4b08-4045-a3b9-ac85370f5e38',
    'lat': 34.043277,
    'lon': -118.447053,
    'is_reserved': 0,
    'is_disabled': 0},
   {'bike_id': '3d1ad672-512d-4430-b27e-e436e7ee1380',
    'lat': 34.042396,
    'lon': -118.256453,
    'is_reserved': 0,
    'is_disabled': 0},
   {'bike_id': 'c75b618c-f1df-45a8-b22c-9e70bc57ccf9',
    'lat': 34.045802506

## Data Wrangling, Part 1
For today's exercises, we are going to be working with data using Python's `pandas` package. We will start with how to read in CSV data, then how to combine multiple dataframes into one. We will then move on to learning about how to calculate summary statistics, join data from different dataframes, and produce a summary extract for export. 

### Exercise 1: Importing & Concatenating Data in Pandas
Yesterday we prepared a series of CSV file extracts containing Metro vehicle location data and saved them to disk. Today we will learn how to read in CSV files using `pandas.read_csv()` instead of the `csv` package. We will then combine multiple dataframes into one dataframe. 

In [191]:
import os

path = 'output'

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory output 


In [37]:
# Import libraries (pandas)
import pandas as pd

# TODO: Add the paths to any two of your CSV files from yesterday
filepath_1 = 'day2-prj/data/raw/lametro_10_2019-09-16-15-46-54.csv'
filepath_2 = 'day2-prj/data/raw/lametro_710_2019-09-16-15-49-56.csv'

# TODO: Read both CSVs like this -> pd.read_csv(filepath)
metro_df1 = pd.read_csv(filepath_1)
metro_df2 = pd.read_csv(filepath_2)

# Here is how we get the number of rows in a dataframe
print(len(metro_df1))
print(len(metro_df2))

12
10


Two dataframes with the same columns can be combined using [pd.concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html). Let's give it a try.

In [38]:
# Concatenate the dataframes
metro_df = pd.concat([metro_df1, metro_df2])

# TODO: Print the length of the new combined dataframe
print(len(metro_df))

22


We've certainly cut down on the amount of code by moving to `pd.read_csv()`. However, do we still need to write out every filepath and read each one in individually? What if we built a loop to read through all of our files and append them together automatically? 
  
Let's take a stab at it. We are going to use Python's [glob](https://docs.python.org/3/library/glob.html) package to search for all of our la metro data extracts. 

In [97]:
import glob

# This handy statement will get us a list of filenames
all_filenames = [i for i in glob.glob('day2-prj/data/raw/lametro*.csv')]

# Create an empty list that will hold our dataframes
df_list = []

# TODO: Loop through all of our `all_filenames`, read_csv to convert to a dataframe, then append the df to df_list
for filename in all_filenames:
    metro_df_i = pd.read_csv(filename)
    print(metro_df_i.shape)
#     print(filename)
    df_list.append(metro_df_i)

# TODO: Concatenate all the dataframes together
# metro_df_all = pd.concat(df_list)

(6, 9)
(6, 9)
(5, 9)
(5, 9)
(5, 9)
(8, 9)
(8, 9)
(9, 9)
(7, 9)
(8, 9)
(17, 9)
(18, 9)
(20, 9)
(21, 9)
(20, 9)
(8, 9)
(7, 9)
(6, 9)
(6, 9)
(6, 9)


In [99]:
len(df_list)

20

In [112]:
metro_df_all = pd.concat(df_list)
metro_df_all.head()

,heading,id,latitude,longitude,predictable,route_id,run_id,seconds_since_report,call_time
0,220.0,3878,34.086086,-118.382164,True,10.0,10_546_0,217,2019-09-17 11:23:28.227052
1,125.0,6016,34.066203,-118.254197,True,10.0,10_546_0,18,2019-09-17 11:23:28.227052
2,269.0,5721,34.083570,-118.323574,True,10.0,10_533_1,19,2019-09-17 11:23:28.227052
3,165.0,5839,34.081272,-118.284447,True,10.0,10_549_0,16,2019-09-17 11:23:28.227052
4,38.0,6139,34.051544,-118.249184,True,10.0,10_533_1,157,2019-09-17 11:23:28.227052


In [113]:
metro_df_all.dropna(subset=['route_id'], inplace=True)

In [114]:
# Let's see how many rows we have in our new SUPER DATAFRAME
print(len(metro_df_all))

195


### Exercise 2: Summarize Collision Data (Fatalities by Year)
 The [Los Angeles Vision Zero Safety Study](https://view.joomag.com/vision-zero-safety-study/0065798001485405769?short), published in January 2017, provides insights about collisions in the City using data from 2009-2013. Let's attempt to replicate some the analysis, which will allow us to begin answering questions such as these: Are we making progress towards that goal? If not, how far behind are we?  

##### Prep
Let's start by reading in the CSV files for collision data. 

In [3]:
# TODO: Import each of the three tables
import pandas as pd

collisions = pd.read_csv('data/raw/Collisions_20092013_SWITRS.csv')
parties = pd.read_csv('data/raw/Party_Tables__Collisions_20092013_SWITRS.csv')
victims = pd.read_csv('data/raw/Victim_Tables__Collisions_20092013_SWITRS.csv')

Let's take a peek at the data we just imported. It is a good habit to build to constantly check the data as you move it through the wrangling process to make sure that it is not getting transformed in unexpected ways. We can take a look at the first couple of rows of a dataframe by using Pandas' [df.head()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) method. 

In [4]:
# Examine the 'head' of the collision table
collisions.head()

,X,Y,OBJECTID,CASE_ID,ACCIDENT_YEAR,PROCDATE,JURIS,COLLISION_DATE,COLLISION_TIME,OFFICER_ID,...,TIMECAT,MONTH_,CITY,COUNTY,STATE,POINT_X,POINT_Y,Match_addr,m_primaryrd,m_secondrd
0,-118.320580,33.989071,4001,4127182,2009,2009-08-22T00:00:00.000Z,1942,2009-01-29T00:00:00.000Z,1215,39053,...,1500,1,LOS ANGELES,LOS ANGELES,CA,-118.320580,33.989071,"W SLAUSON AVE & 3RD AVE, LOS ANGELES, CA, 90043",W SLAUSON AVE,3RD AVE
1,-118.262935,34.044239,4002,4127195,2009,2009-08-25T00:00:00.000Z,1942,2009-01-23T00:00:00.000Z,1320,37506,...,1500,1,LOS ANGELES,LOS ANGELES,CA,-118.262935,34.044239,"S FLOWER ST & W OLYMPIC BLVD, LOS ANGELES, CA,...",S FLOWER ST,W OLYMPIC BLVD
2,-118.254034,34.084091,4003,4127211,2009,2009-09-08T00:00:00.000Z,1942,2009-01-17T00:00:00.000Z,1250,37506,...,1500,1,LOS ANGELES,LOS ANGELES,CA,-118.254034,34.084091,"ECHO PARK AVE & EFFIE ST, LOS ANGELES, CA, 90026",ECHO PARK AVE,EFFIE ST
3,-118.177198,34.079670,4004,4127215,2009,2009-09-09T00:00:00.000Z,1942,2009-01-27T00:00:00.000Z,305,36814,...,600,1,LOS ANGELES,LOS ANGELES,CA,-118.177198,34.079670,"PHELPS AVE & LYNNFIELD ST, LOS ANGELES, CA, 90032",PHELPS AVE,LYNNFIELD ST
4,-118.257480,34.050159,4005,4127220,2009,2009-09-08T00:00:00.000Z,1942,2009-01-30T00:00:00.000Z,1400,39163,...,1500,1,LOS ANGELES,LOS ANGELES,CA,-118.257480,34.050159,"W 6TH ST & S FLOWER ST, LOS ANGELES, CA, 90017",W 6TH ST,S FLOWER ST


What do all of these feature values represent? You can take a look at the [SWITRS Codebook](https://peteraldhous.com/Data/ca_traffic/SWITRS_codebook.pdf), which will explain each variable in all three of our tables.

In [5]:
# Alternatively, we can look at the 'tail' of the table by running the code below
parties.tail()

,OBJECTID,CASE_ID,PARTY_NUMBER,PARTY_TYPE,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,...,VEHICLE_YEAR,VEHICLE_MAKE,STWD_VEHICLE_TYPE,CHP_VEH_TYPE_TOWING,CHP_VEH_TYPE_TOWED,RACE,INATTENTION,SPECIAL_INFO_F,SPECIAL_INFO_G,ACCIDENT_YEAR
937667,936996,9018771,1,1.0,Y,M,19,B,-,S,...,2002.0,NISSAN,-,-,00,O,NaN,-,-,2006
937668,936997,9018771,2,1.0,N,M,50,A,-,S,...,1987.0,CHEVROLET,-,-,00,W,NaN,-,-,2006
937669,936998,9018771,3,1.0,N,M,58,A,-,N,...,2005.0,HONDA,-,-,00,W,NaN,-,-,2006
937670,936999,9018771,4,1.0,N,F,57,A,-,N,...,1998.0,MERCURY,-,-,00,B,NaN,-,-,2006
937671,937000,9018773,1,1.0,Y,M,40,A,-,W,...,2004.0,FORD,-,-,00,W,NaN,-,-,2006


In [6]:
 # TODO: Change the size of the head to show the top 15 rows (see documentation above)
parties.head(15)

,OBJECTID,CASE_ID,PARTY_NUMBER,PARTY_TYPE,AT_FAULT,PARTY_SEX,PARTY_AGE,PARTY_SOBRIETY,PARTY_DRUG_PHYSICAL,DIR_OF_TRAVEL,...,VEHICLE_YEAR,VEHICLE_MAKE,STWD_VEHICLE_TYPE,CHP_VEH_TYPE_TOWING,CHP_VEH_TYPE_TOWED,RACE,INATTENTION,SPECIAL_INFO_F,SPECIAL_INFO_G,ACCIDENT_YEAR
0,9001,670821,1,1.0,Y,M,39,A,-,W,...,2002.0,LEXUS,A,NaN,NaN,H,NaN,-,-,2003
1,9002,670823,1,1.0,Y,F,22,A,-,S,...,1997.0,FORD,A,NaN,NaN,H,NaN,-,-,2003
2,9003,670823,2,1.0,N,M,22,A,-,W,...,1986.0,HONDA,A,NaN,NaN,H,NaN,-,-,2003
3,9004,670824,1,1.0,Y,-,998,G,G,W,...,0.0,-,A,NaN,NaN,NaN,NaN,-,-,2003
4,9005,670824,2,1.0,N,F,68,A,-,N,...,1994.0,MAZDA,A,NaN,NaN,O,NaN,-,-,2003
5,9006,670825,1,1.0,Y,-,998,G,G,N,...,0.0,OTHER - AUTO,A,NaN,NaN,NaN,NaN,-,-,2003
6,9007,670825,2,2.0,N,F,16,A,-,W,...,0.0,-,N,NaN,NaN,H,NaN,-,-,2003
7,9008,670825,3,2.0,N,F,17,A,-,W,...,0.0,-,N,NaN,NaN,H,NaN,-,-,2003
8,9009,670825,4,2.0,N,F,15,A,-,W,...,0.0,-,N,NaN,NaN,H,NaN,-,-,2003
9,9010,670826,1,1.0,N,M,67,A,-,E,...,2001.0,CHEVROLET,A,NaN,NaN,W,NaN,-,-,2003


When we are accessing really WIDE tables like the ones above, the `df.head()` method won't show us all the column names. We can access the column names by calling `df.columns` to get the list of all columns in the table. Access the [SWITRS Codebook](https://peteraldhous.com/Data/ca_traffic/SWITRS_codebook.pdf) for detail on what all these column values represent.

In [7]:
# TODO: Print the column names for the collisions table
collisions.columns

Index(['X', 'Y', 'OBJECTID', 'CASE_ID', 'ACCIDENT_YEAR', 'PROCDATE', 'JURIS',
       'COLLISION_DATE', 'COLLISION_TIME', 'OFFICER_ID', 'REPORTING_DISTRICT',
       'DAY_OF_WEEK', 'SHIFT', 'POPULATION', 'CNTY_CITY_LOC', 'SPECIAL_COND',
       'BEAT_TYPE', 'CHP_BEAT_TYPE', 'CITY_DIVISION_LAPD', 'CHP_BEAT_CLASS',
       'BEATNUMB', 'PRIMARY_RD', 'SECONDARY_RD', 'DISTANCE', 'DIRECTION',
       'INTERSECTION', 'WEATHER_1', 'WEATHER_2', 'STATE_HWY_IND',
       'CALTRANS_COUNTY', 'CALTRANS_DISTRICT', 'STATE_ROUTE', 'ROUTE_SUFFIX',
       'POSTMILE_PREFIX', 'POSTMILE', 'LOCATION_TYPE', 'RAMP_INTERSECTION',
       'SIDE_OF_HWY', 'TOW_AWAY', 'COLLISION_SEVERITY', 'NUMBER_KILLED',
       'NUMBER_INJURED', 'PARTY_COUNT', 'PRIMARY_COLL_FACTOR', 'PCF_CODE_VIOL',
       'PCF_VIOL_CATEGORY', 'PCF_VIOLATION', 'PCF_VIOL_SUBSECTION',
       'HIT_AND_RUN', 'TYPE_OF_COLLISION', 'MVIW', 'PED_ACTION',
       'ROAD_SURFACE', 'ROAD_COND_1', 'ROAD_COND_2', 'LIGHTING',
       'CONTROL_DEVICE', 'CHP_ROAD_TYPE', '

How many rows are in each dataset? For each of the tables, print out the following: `The number of rows in the [table name] is [number of rows]`. For example: `The number of rows in the collision table is 1200`. 

In [8]:
# TODO: Print out the number of rows for each of the three datafram



for table_name in [collisions, parties, victims]:
    row_count = table_name.shape[0]
    print(row_count)

print(f'The number of rows in the collision table is {collisions.shape[0]}')
print(f'The number of rows in the party table is {parties.shape[0]}')
print(f'The number of rows in the victim table is {victims.shape[0]}')

173714
937672
576617
The number of rows in the collision table is 173714
The number of rows in the party table is 937672
The number of rows in the victim table is 576617


##### Step 1: Filter for fatalities
The first step to summarazing fatalites by year is to employ a row-based filter to our data.



In [9]:
# TODO: Filter for only collisions
fatal_collisions = collisions[collisions['COLLISION_SEVERITY'] == 1]

In [15]:
len(collisions.query('COLLISION_SEVERITY == 1'))

875

How many fatal collisions occurred between 2009 and 2013?

In [17]:
# TODO: Print out the number of fatal collisions (Hint: It is the rowcount, since we filtered for only fatal collisions)
fatal_collisions.shape[0]

875

##### Step 2: Count fatalities by year
An important distinction to be aware of when reading the report and replicating the findings is whether the statistic refers to _collisons_ or to _victims_ since this will determine which table we will be working with. In this case, since we are benchmarking our progress against the goal of zero _fatalities_ we will be looking at the victim table. 

In [18]:
victims.columns

Index(['OBJECTID', 'CASE_ID', 'PARTY_NUMBER', 'VICTIM_ROLE', 'VICTIM_SEX',
       'VICTIM_AGE', 'VICTIM_DEGREE_OF_INJURY', 'VICTIM_SEATING_POSITION',
       'VICTIM_SAFETY_EQUIP_1', 'VICTIM_SAFETY_EQUIP_2', 'VICTIM_EJECTED',
       'ACCIDENT_YEAR'],
      dtype='object')

In [19]:
# TODO: Within fatal_collisions, group by year and count
fatalities_victim = victims[victims['VICTIM_DEGREE_OF_INJURY'] == 1]

fatalities_by_year = fatalities_victim.ACCIDENT_YEAR.value_counts().sort_index()

# Let's take a look
print(fatalities_by_year)

2003    242
2004    221
2005    203
2006    227
2007    210
2008    227
2009    181
2010    184
2011    157
2012    200
2013    201
2014    196
Name: ACCIDENT_YEAR, dtype: int64


### Exercise 3: Prepare and Export a Custom Data Extract
While the Jupyter notebook is in most cases the ideal place to explore and present data, we occassionaly want to be able to export data or findings for use elsewhere. After we take a look at the trends for traffic fatalities, we are going to prepare a custom data extract.

It is not uncommon for policymakers to want to know more specific information for those who have been killed in a traffic collision. Let's pretend the Mayor has asked for a list of all fatalities in the year 2013 with the following information for each fatality:
1. Date & time of collision - Collision Table
2. Type of collision (Head-On, Sideswipe, etc.) - Collision Table
3. Whether Alcohol was involved (yes/no) - Collision Table
4. Age of the victim - Victim Table
5. Victim's mode of transportation - Victim Table
6. Distance of the collision from the intersection (to know whether it was intersection-related) - Collision Table

Let's produce this data extract.

##### Step 1: Filter for 2013 fatalities
We can begin this assignment by noting that we are only interested in fatalities within 2013. We will therefore be applying 2 filters:  
- Year = 2013 and 
- Victim Degree of Injury = 1 (fatal)

In [20]:
# TODO: Filter for 2013 victims
victims_2013 = victims[victims['ACCIDENT_YEAR'] == 2013]

# TODO: Filter 2013 fatal victims 
fatal_victims_2013 = victims_2013[victims_2013['VICTIM_DEGREE_OF_INJURY'] == 1]

# Examine what's left
fatal_victims_2013.head()

,OBJECTID,CASE_ID,PARTY_NUMBER,VICTIM_ROLE,VICTIM_SEX,VICTIM_AGE,VICTIM_DEGREE_OF_INJURY,VICTIM_SEATING_POSITION,VICTIM_SAFETY_EQUIP_1,VICTIM_SAFETY_EQUIP_2,VICTIM_EJECTED,ACCIDENT_YEAR
495785,498786,5946386,2,3,M,50,1,9.0,-,-,3.0,2013
496320,496321,5639079,2,3,M,55,1,9.0,-,-,3.0,2013
496321,496322,5639091,2,3,M,23,1,9.0,-,-,3.0,2013
496322,496323,5639301,1,1,F,44,1,1.0,L,H,0.0,2013
496323,496324,5639357,1,1,M,29,1,1.0,P,W,1.0,2013


How many fatalities were in 2013? How is this number different from the count of fatal _collisions?_

In [21]:
# TODO: Print out the number of total number of victims of fatal injury for 2013 
fatal_victims_2013.shape[0]

201

In [22]:
fatal_collisions_2013 = fatal_collisions[fatal_collisions['ACCIDENT_YEAR'] == 2013]
fatal_collisions_2013.shape[0]

190

##### Step 2: Join tables  
Looking through the SWITRS Codebook, you will notice that these data are not contained within a single table - instead we are going to need to join at least 2 tables to produce the necessary information. For each of the requested fields, find it in the codebook to determine the table the field resides in; you will then want to make an **_inner join_** (see [here](http://www.khankennels.com/blog/index.php/archives/2007/04/20/getting-joins/) for a summary of join types) with the currently filtered fatalies data.  

![joins](join_types.png)

In [23]:
collisions.columns

Index(['X', 'Y', 'OBJECTID', 'CASE_ID', 'ACCIDENT_YEAR', 'PROCDATE', 'JURIS',
       'COLLISION_DATE', 'COLLISION_TIME', 'OFFICER_ID', 'REPORTING_DISTRICT',
       'DAY_OF_WEEK', 'SHIFT', 'POPULATION', 'CNTY_CITY_LOC', 'SPECIAL_COND',
       'BEAT_TYPE', 'CHP_BEAT_TYPE', 'CITY_DIVISION_LAPD', 'CHP_BEAT_CLASS',
       'BEATNUMB', 'PRIMARY_RD', 'SECONDARY_RD', 'DISTANCE', 'DIRECTION',
       'INTERSECTION', 'WEATHER_1', 'WEATHER_2', 'STATE_HWY_IND',
       'CALTRANS_COUNTY', 'CALTRANS_DISTRICT', 'STATE_ROUTE', 'ROUTE_SUFFIX',
       'POSTMILE_PREFIX', 'POSTMILE', 'LOCATION_TYPE', 'RAMP_INTERSECTION',
       'SIDE_OF_HWY', 'TOW_AWAY', 'COLLISION_SEVERITY', 'NUMBER_KILLED',
       'NUMBER_INJURED', 'PARTY_COUNT', 'PRIMARY_COLL_FACTOR', 'PCF_CODE_VIOL',
       'PCF_VIOL_CATEGORY', 'PCF_VIOLATION', 'PCF_VIOL_SUBSECTION',
       'HIT_AND_RUN', 'TYPE_OF_COLLISION', 'MVIW', 'PED_ACTION',
       'ROAD_SURFACE', 'ROAD_COND_1', 'ROAD_COND_2', 'LIGHTING',
       'CONTROL_DEVICE', 'CHP_ROAD_TYPE', '

In [24]:
parties.columns

Index(['OBJECTID', 'CASE_ID', 'PARTY_NUMBER', 'PARTY_TYPE', 'AT_FAULT',
       'PARTY_SEX', 'PARTY_AGE', 'PARTY_SOBRIETY', 'PARTY_DRUG_PHYSICAL',
       'DIR_OF_TRAVEL', 'PARTY_SAFETY_EQUIP_1', 'PARTY_SAFETY_EQUIP_2',
       'FINAN_RESPONS', 'SP_INFO_1', 'SP_INFO_2', 'SP_INFO_3',
       'OAF_VIOLATION_CODE', 'OAF_VIOL_CAT', 'OAF_VIOL_SECTION',
       'OAF_VIOLATION_SUFFIX', 'OAF_1', 'OAF_2', 'PARTY_NUMBER_KILLED',
       'PARTY_NUMBER_INJURED', 'MOVE_PRE_ACC', 'VEHICLE_YEAR', 'VEHICLE_MAKE',
       'STWD_VEHICLE_TYPE', 'CHP_VEH_TYPE_TOWING', 'CHP_VEH_TYPE_TOWED',
       'RACE', 'INATTENTION', 'SPECIAL_INFO_F', 'SPECIAL_INFO_G',
       'ACCIDENT_YEAR'],
      dtype='object')

In [25]:
victims.columns

Index(['OBJECTID', 'CASE_ID', 'PARTY_NUMBER', 'VICTIM_ROLE', 'VICTIM_SEX',
       'VICTIM_AGE', 'VICTIM_DEGREE_OF_INJURY', 'VICTIM_SEATING_POSITION',
       'VICTIM_SAFETY_EQUIP_1', 'VICTIM_SAFETY_EQUIP_2', 'VICTIM_EJECTED',
       'ACCIDENT_YEAR'],
      dtype='object')

In [26]:
# TODO: Join the necessary tables using pd.merge
merged_fatal_victims_2013 = pd.merge(fatal_collisions_2013, fatal_victims_2013, on = 'CASE_ID')
merged_fatal_victims_2013 = pd.merge(merged_fatal_victims_2013, parties, on = ['CASE_ID','PARTY_NUMBER'])


In [27]:
merged_fatal_victims_2013.shape[0]

201

##### Step 3: Filter out un-necessary columns
After joining both of the tables, you will notice that we end up with a large number of columns. We don't want to make the policymakers hunt through the data extract for the necessary information; instead, we should be filtering for only the columns that were requested in the extract.

In [28]:
# TODO: Let's make a list of all the columns we want to keep
keep_cols = ['CASE_ID' , 'COLLISION_DATE' , 'COLLISION_TIME' , 'TYPE_OF_COLLISION' , 
             'ALCOHOL_INVOLVED' , 'VICTIM_AGE' , 'VICTIM_ROLE' , 'DISTANCE', 'PARTY_TYPE']

# TODO: Filter the df for those columns we want
output_fatal_victims_2013 = merged_fatal_victims_2013[keep_cols]

In [29]:
output_fatal_victims_2013.head()

,CASE_ID,COLLISION_DATE,COLLISION_TIME,TYPE_OF_COLLISION,ALCOHOL_INVOLVED,VICTIM_AGE,VICTIM_ROLE,DISTANCE,PARTY_TYPE
0,5639079,2013-01-05T00:00:00.000Z,2010,G,Y,55,3,60,2.0
1,5639091,2013-01-01T00:00:00.000Z,255,G,Y,23,3,123,2.0
2,5639301,2013-01-08T00:00:00.000Z,130,E,NaN,44,1,0,1.0
3,5639357,2013-01-12T00:00:00.000Z,2210,E,Y,29,1,100,1.0
4,5639358,2013-01-10T00:00:00.000Z,1135,B,NaN,71,1,163,1.0


##### Step 4: Write-out the CSV
We can write out a CSV using the syntax: `df.to_csv(filepath)`.

In [30]:
# Before writing out, let's confirm that our current df matches the desired output format,
# using panda's .head() method from before
output_fatal_victims_2013.head()

,CASE_ID,COLLISION_DATE,COLLISION_TIME,TYPE_OF_COLLISION,ALCOHOL_INVOLVED,VICTIM_AGE,VICTIM_ROLE,DISTANCE,PARTY_TYPE
0,5639079,2013-01-05T00:00:00.000Z,2010,G,Y,55,3,60,2.0
1,5639091,2013-01-01T00:00:00.000Z,255,G,Y,23,3,123,2.0
2,5639301,2013-01-08T00:00:00.000Z,130,E,NaN,44,1,0,1.0
3,5639357,2013-01-12T00:00:00.000Z,2210,E,Y,29,1,100,1.0
4,5639358,2013-01-10T00:00:00.000Z,1135,B,NaN,71,1,163,1.0


In [31]:
# TODO: Write out the CSV to the data/processed directory
output_fatal_victims_2013.to_csv('output/fatal_victims_2013.csv' , index = False, sep = ',')

### Bonus Challenge Exercise
Prepare the same extract, but this time include the following additional fields:
- Whether the victim was at fault...